In [ ]:
# Question: Create a Flask app that displays "Hello, World!" on the homepage.

!pip install flask

from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
  return "Hello, World!"

if __name__ == "__main__":
  app.run()


In [ ]:
# Question: Build a Flask app with static HTML pages and navigate between them.

from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def index():
  return render_template('index.html')

@app.route('/about')
def about():
  return render_template('about.html')

if __name__ == '__main__':
  app.run(debug=True)


In [ ]:
# Question: Develop a Flask app that uses URL parameters to display dynamic content.

@app.route('/user/<username>')
def show_user_profile(username):
  # Replace with actual user data retrieval
  user_info = {'name': username, 'bio': 'A Flask enthusiast'}
  return render_template('user_profile.html', user=user_info)


In [ ]:
# Question: 4. Create a Flask app with a form that accepts user input and displays it.

from flask import request

@app.route('/submit', methods=['GET', 'POST'])
def submit():
  if request.method == 'POST':
    user_input = request.form['user_input']
    return render_template('display_input.html', input=user_input)
  return render_template('input_form.html')


In [ ]:
# Question: 5. Implement user sessions in a Flask app to store and display user-specific data.

!pip install flask-session

from flask import Flask, session
from flask_session import Session

app = Flask(__name__)
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

@app.route('/set_session/<data>')
def set_session(data):
    session['user_data'] = data
    return f"Session data set to: {data}"

@app.route('/get_session')
def get_session():
    if 'user_data' in session:
        return f"Stored session data: {session['user_data']}"
    return "No session data found"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.3/210.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
# Question: 6. Build a Flask app that allows users to upload files and display them on the website.

from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)

UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/', methods=['GET', 'POST'])
def upload_file():
  if request.method == 'POST':
    if 'file' not in request.files:
      return 'No file part'
    file = request.files['file']
    if file.filename == '':
      return 'No selected file'
    if file:
      filename = secure_filename(file.filename)
      file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
      return render_template('display_file.html', filename=filename)
  return render_template('upload_form.html')

@app.route('/uploads/<filename>')
def display_uploaded_file(filename):
  return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

if __name__ == '__main__':
  app.run(debug=True)


In [ ]:
# Question: Integrate a SQLite database with Flask to perform CRUD operations on a list of items.

import sqlite3
from flask import Flask, render_template, request, redirect

app = Flask(__name__)
DATABASE = 'items.db'

def get_db():
  db = getattr(g, '_database', None)
  if db is None:
    db = g._database = sqlite3.connect(DATABASE)
  return db

def create_table():
  with app.app_context():
    db = get_db()
    cursor = db.cursor()
    cursor.execute('''
      CREATE TABLE IF NOT EXISTS items (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL
      )
    ''')
    db.commit()

@app.teardown_appcontext
def close_connection(exception):
  db = getattr(g, '_database', None)
  if db is not None:
    db.close()

@app.route('/')
def index():
  with app.app_context():
    db = get_db()
    cursor = db.cursor()
    cursor.execute('SELECT * FROM items')
    items = cursor.fetchall()
    return render_template('item_list.html', items=items)

@app.route('/add', methods=['POST'])
def add_item():
  name = request.form['name']
  with app.app_context():
    db = get_db()
    cursor = db.cursor()
    cursor.execute('INSERT INTO items (name) VALUES (?)', (name,))
    db.commit()
  return redirect('/')

@app.route('/edit/<int:item_id>')
def edit_item(item_id):
  with app.app_context():
    db = get_db()
    cursor = db.cursor()
    cursor.execute('SELECT * FROM items WHERE id = ?', (item_id,))
    item = cursor.fetchone()
    return render_template('edit_item.html', item=item)

@app.route('/update/<int:item_id>', methods=['POST'])
def update_item(item_id):
  name = request.form['name']
  with app.app_context():
    db = get_db()
    cursor = db.cursor()
    cursor.execute('UPDATE items SET name = ? WHERE id = ?', (name, item_id))
    db.commit()
  return redirect('/')

@app.route('/delete/<int:item_id>')
def delete_item(item_id):
  with app.app_context():
    db = get_db()
    cursor = db.cursor()
    cursor.execute('DELETE FROM items WHERE id = ?', (item_id,))
    db.commit()
  return redirect('/')

if __name__ == '__main__':
  create_table()
  app.run(debug=True)


In [ ]:
# Question: 8. Implement user authentication and registration in a Flask app using Flask-Login.

!pip install flask-login

from flask import Flask, render_template, request, redirect, url_for
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required, current_user
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
app.secret_key = 'your_secret_key'  # Replace with a strong secret key

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'

# In-memory user database (replace with a real database for production)
users = {}

class User(UserMixin):
  def __init__(self, id, username, password):
    self.id = id
    self.username = username
    self.password = password

@login_manager.user_loader
def load_user(user_id):
  return users.get(int(user_id))

@app.route('/register', methods=['GET', 'POST'])
def register():
  if request.method == 'POST':
    username = request.form['username']
    password = request.form['password']
    hashed_password = generate_password_hash(password)

    # Check if username already exists
    if username in users:
      return 'Username already exists'

    # Create new user and add to database
    new_user = User(len(users) + 1, username, hashed_password)
    users[new_user.id] = new_user

    return redirect(url_for('login'))
  return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
  if request.method == 'POST':
    username = request.form['username']
    password = request.form['password']

    user = None
    for u in users.values():
      if u.username == username:
        user = u
        break

    if user and check_password_hash(user.password, password):
      login_user(user)
      return redirect(url_for('protected'))
    else:
      return 'Invalid username or password'

  return render_template('login.html')

@app.route('/logout')
@login_required
def logout():
  logout_user()
  return redirect(url_for('login'))

@app.route('/protected')
@login_required
def protected():
  return f'Hello, {current_user.username}! This is a protected page.'

if __name__ == '__main__':
  app.run(debug=True)


In [ ]:
# Question: 9. Create a RESTful API using Flask to perform CRUD operations on resources like books or movies.

from flask import Flask, jsonify, request
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///movies.db'  # Replace with your preferred database
db = SQLAlchemy(app)

class Movie(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(80), nullable=False)
    genre = db.Column(db.String(50))
    release_year = db.Column(db.Integer)

    def __repr__(self):
        return f'<Movie {self.title}>'

db.create_all()

# CREATE
@app.route('/movies', methods=['POST'])
def create_movie():
    data = request.get_json()
    new_movie = Movie(title=data['title'], genre=data.get('genre'), release_year=data.get('release_year'))
    db.session.add(new_movie)
    db.session.commit()
    return jsonify({'message': 'Movie created', 'movie_id': new_movie.id}), 201

# READ (all)
@app.route('/movies', methods=['GET'])
def get_movies():
    movies = Movie.query.all()
    movie_list = [{'id': movie.id, 'title': movie.title, 'genre': movie.genre, 'release_year': movie.release_year} for movie in movies]
    return jsonify(movie_list)

# READ (one)
@app.route('/movies/<int:movie_id>', methods=['GET'])
def get_movie(movie_id):
    movie = Movie.query.get(movie_id)
    if movie:
        return jsonify({'id': movie.id, 'title': movie.title, 'genre': movie.genre, 'release_year': movie.release_year})
    return jsonify({'message': 'Movie not found'}), 404

# UPDATE
@app.route('/movies/<int:movie_id>', methods=['PUT'])
def update_movie(movie_id):
    movie = Movie.query.get(movie_id)
    if movie:
        data = request.get_json()
        movie.title = data.get('title', movie.title)
        movie.genre = data.get('genre', movie.genre)
        movie.release_year = data.get('release_year', movie.release_year)
        db.session.commit()
        return jsonify({'message': 'Movie updated'})
    return jsonify({'message': 'Movie not found'}), 404

# DELETE
@app.route('/movies/<int:movie_id>', methods=['DELETE'])
def delete_movie(movie_id):
    movie = Movie.query.get(movie_id)
    if movie:
        db.session.delete(movie)
        db.session.commit()
        return jsonify({'message': 'Movie deleted'})
    return jsonify({'message': 'Movie not found'}), 404

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# Question: 10. Design a Flask app with proper error handling for 404 and 500 errors.

app = Flask(__name__)

# ... (rest of your app code)

@app.errorhandler(404)
def page_not_found(error):
  return render_template('404.html'), 404

@app.errorhandler(500)
def internal_server_error(error):
  return render_template('500.html'), 500


In [ ]:
# Question: Create a real-time chat application using Flask-SocketIO.

!pip install flask-socketio

from flask import Flask, render_template
from flask_socketio import SocketIO, emit

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app)

@app.route('/')
def index():
  return render_template('chat_index.html')

@socketio.on('message')
def handle_message(message):
  emit('message', message, broadcast=True)

if __name__ == '__main__':
  socketio.run(app, debug=True)


In [ ]:
# Question: 12. Build a Flask app that updates data in real-time using WebSocket connections.

!pip install flask-socketio

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app)

@app.route('/')
def index():
  return render_template('index.html')

@socketio.on('connect')
def handle_connect():
  print('Client connected')

@socketio.on('disconnect')
def handle_disconnect():
  print('Client disconnected')

@socketio.on('data_update')
def handle_data_update(data):
  # Process the received data (e.g., update a database, perform calculations)
  # ...

  # Emit an event to update clients with the processed data
  emit('updated_data', processed_data, broadcast=True)

if __name__ == '__main__':
  socketio.run(app, debug=True)


In [ ]:
# Question: Implement notifications in a Flask app using websockets to notify users of updates.

!pip install flask-socketio

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app)

@app.route('/')
def index():
  return render_template('index.html')  # Assuming you have an index.html

@socketio.on('connect')
def handle_connect():
  print('Client connected')

# Trigger this function whenever you want to send a notification
def send_notification(message, user_id=None):
  if user_id:
    # Send to a specific user using their SocketIO session ID
    emit('notification', message, room=user_id)
  else:
    # Broadcast to all connected users
    emit('notification', message, broadcast=True)

if __name__ == '__main__':
  socketio.run(app, debug=True)
